In [ ]:
import pandas as pd
import lecilab_behavior_analysis.utils as utils
from pathlib import Path
%load_ext autoreload
%autoreload 2

In [ ]:
# mouse = "mouse2"
# df = utils.load_example_data(mouse)

In [ ]:
# load data from cluster
tv_projects = utils.get_server_projects()
print(tv_projects)

In [ ]:
# see the available animals
animals = utils.get_animals_in_project(tv_projects[1])
print(animals)

In [ ]:
# download the data for a specific animal

# mouse = "mouse2"
# df = utils.load_example_data(mouse)

mouse = "ACV001"
local_path = Path(utils.get_outpath()) / Path(tv_projects[1]) / Path("sessions") / Path(mouse)
# create the directory if it doesn't exist
local_path.mkdir(parents=True, exist_ok=True)
# download the session data
utils.rsync_cluster_data(
    project_name=tv_projects[1],
    file_path="sessions/{}/{}.csv".format(mouse, mouse),
    local_path=str(local_path),
    credentials=utils.get_idibaps_cluster_credentials(),
)

In [ ]:
# load the data
df = pd.read_csv(local_path / Path(f'{mouse}.csv'), sep=";")

In [ ]:
# # engagement analysis
# from lecilab_behavior_analysis import df_transforms as dft
# df = dft.add_trial_duration_column_to_df(df)
# df = dft.add_engagement_column(df, engagement_sd_criteria=2)
# # load events
# events_df = utils.load_all_events(project_name=tv_projects[1])
# sbu_df = dft.get_box_usage_df(df, events_df)
# sbu_df = dft.add_day_column_to_df(sbu_df)

In [ ]:
# from lecilab_behavior_analysis import plots
# plots.plot_box_usage_by_date(sbu_df)

In [ ]:
from lecilab_behavior_analysis.figure_maker import subject_progress_figure
# sync and load events
for file in ["old_events", "events.csv"]:
    utils.rsync_cluster_data(
        project_name=tv_projects[1],
        file_path=file,
        local_path=str(local_path),
        credentials=utils.get_idibaps_cluster_credentials(),
    )
events_df = utils.load_all_events(project_name=tv_projects[1])

In [ ]:
fig = subject_progress_figure(df.head(10000), perf_window=100, summary_matrix_plot=False, events_df=events_df)

In [ ]:
# # load the data if cluster is down
# mouse_path = utils.get_outpath() + "/visual_and_COT_data/sessions/ACV008/ACV008.csv"
# df = pd.read_csv(mouse_path, sep=";")

In [ ]:
from lecilab_behavior_analysis.figure_maker import session_summary_figure
from lecilab_behavior_analysis.df_transforms import add_trial_of_day_column_to_df, add_day_column_to_df
import numpy as np
# select the session you want to plot
date = "2025-07-24"
# df = analyze_df(df)
df = add_day_column_to_df(df)
df = add_trial_of_day_column_to_df(df)
df['total_trial'] = np.arange(1, df.shape[0] + 1)
sdf = df[df["year_month_day"] == date]
# sdf = df[df["current_training_stage"] == "TwoAFC_auditory_hard"]
fig = session_summary_figure(sdf, perf_window=50, width=15, height=10)

In [ ]:
from lecilab_behavior_analysis import df_transforms as dft
from lecilab_behavior_analysis import utils
sdf = utils.add_time_from_session_start(sdf)

In [ ]:
sdf[sdf.session != sdf.session.shift(1)].index

In [ ]:
sdf = dft.adjust_trials_and_time_of_start_to_first_session(sdf)

In [ ]:
from lecilab_behavior_analysis import plots
# find the index of the session changes
session_changes = sdf[sdf.session != sdf.session.shift(1)].index
plots.plot_trial_time_of_start(sdf, session_changes=session_changes, width=15, height=10)